# explore this [tutorial](https://plainenglish.io/blog/a-simple-guide-to-plotly-for-plotting-financial-chart-54986c996682)
- The full code and demo notebook of this tutorial is available[here](https://github.com/yongghongg/stock-screener/blob/main/plotly_financial_chart_demo.ipynb)

> [!HINT] FROM HERE
> https://pypi.org/project/yfinance/

## [get historical market data](https://datascience.stackexchange.com/questions/55811/getting-stock-data-in-a-discipline-manner-from-yahoo-finance)

add !pip install plotly==5.2.1

2.nd version

## In the next this section, we will try to add volume, MACD, and stochastic as subplots.

- [FROM HERE](https://github.com/yongghongg/stock-screener/blob/main/plotly_financial_chart_demo.ipynb)

In [ ]:
# use python version 3.10.12

# install necessary modules
# %pip install --user yfinance==0.2.40
%pip install --user plotly==5.22.0
%pip install --user ta==0.11.0
%pip install --user plotly.express

import pandas as pd
import yfinance as yf
from datetime import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from ta.trend import MACD
from ta.momentum import StochasticOscillator
# FROM HERE
# https://stackoverflow.com/questions/75441104/why-is-rsi-calculation-with-python-ta-library-changes-depending-on-starting-posi
from ta.momentum import RSIIndicator


# cache requests_cache
# https://pypi.org/project/yfinance/

# download stock price data
# symbol = 'TREX'
# df = yf.download(symbol, start='2022-01-01')

# load data from local file
# df => DataFrame
df = pd.read_csv('TREX.csv', index_col=0)

# [*********************100%***********************]  1 of 1 completed

# https://medium.datadriveninvestor.com/python-relative-strength-index-rsi-calculation-and-visualization-c44e4671bf1c


# fig = go.Figure(go.Candlestick(x=df.index,
#    open=df['Open'],
#    high=df['High'],
#    low=df['Low'],
#    close=df['Close']))

# fig.show()

# removing rangeslider
# fig.update_layout(xaxis_rangeslider_visible=False)

# removing all empty dates
# build complete timeline from start date to end date
dt_all = pd.date_range(start=df.index[0],end=df.index[-1])
# retrieve the dates that ARE in the original dataset
dt_obs = [d.strftime("%Y-%m-%d") for d in pd.to_datetime(df.index)]
# define dates with missing values
dt_breaks = [d for d in dt_all.strftime("%Y-%m-%d").tolist() if not d in dt_obs]

# # add moving averages to df
df['MA20'] = df['Close'].rolling(window=20).mean()
df['MA5'] = df['Close'].rolling(window=5).mean()
# Calculate the 50-day and 200-day simple moving averages
# FROM HERE
# https://fastercapital.com/keyword/yf-download-historical-data.html
df["SMA50"] = df["Close"].rolling(50).mean()
df["SMA200"] = df["Close"].rolling(200).mean()

# MACD
macd = MACD(close=df['Close'], 
             window_slow=26,
             window_fast=12, 
             window_sign=9)
 # stochastics
stoch = StochasticOscillator(high=df['High'],
                              close=df['Close'],
                              low=df['Low'],
                              window=14, 
                              smooth_window=3)
rsi = RSIIndicator(close=df['Close']).rsi()

# add subplot properties when initiliazing fig variable
fig = make_subplots(rows=5, cols=1, shared_xaxes=True,
                    vertical_spacing=0.01, 
                    row_heights=[0.5,0.3,0.5,0.3,0.3])
# Plot OHLC on 1st subplot (using the codes from before)
fig.add_trace(go.Candlestick(x=df.index,
                             open=df['Open'],
                             high=df['High'],
                             low=df['Low'],
                             close=df['Close'], 
                             showlegend=False))
# add moving average traces
fig.add_trace(go.Scatter(x=df.index, 
                         y=df['SMA200'], 
                         line=dict(color='green', width=2), 
                         name='SMA200'))
fig.add_trace(go.Scatter(x=df.index, 
                         y=df['SMA50'], 
                         line=dict(color='brown', width=2), 
                         name='SMA50'))
fig.add_trace(go.Scatter(x=df.index, 
                         y=df['MA5'], 
                         line=dict(color='blue', width=2), 
                         name='MA 5'))
fig.add_trace(go.Scatter(x=df.index, 
                         y=df['MA20'], 
                         line=dict(color='orange', width=2), 
                         name='MA 20'))

# Plot volume trace on 2nd row 
colors = ['green' if row['Open'] - row['Close'] >= 0 
          else 'red' for index, row in df.iterrows()]
fig.add_trace(go.Bar(x=df.index, 
                     y=df['Volume'],
                     marker_color=colors,
                     name="Volume",
                    ), row=2, col=1)

# Plot MACD trace on 3rd row
colors = ['green' if val >= 0 
          else 'red' for val in macd.macd_diff()]
fig.add_trace(go.Bar(x=df.index, 
                     y=macd.macd_diff(),
                     marker_color=colors,
                     name="MACD diff",
                    ), row=3, col=1)
fig.add_trace(go.Scatter(x=df.index,
                         y=macd.macd(),
                         line=dict(color='black', width=2),
                         name="MACD",
                        ), row=3, col=1)
fig.add_trace(go.Scatter(x=df.index,
                         y=macd.macd_signal(),
                         line=dict(color='blue', width=1),
                         name="MACD_SIGNAL",
                        ), row=3, col=1)

# Plot stochastics trace on 4th row 
fig.add_trace(go.Scatter(x=df.index,
                         y=stoch.stoch(),
                         line=dict(color='black', width=2),
                         name="stochastic",
                        ), row=4, col=1)
fig.add_trace(go.Scatter(x=df.index,
                         y=stoch.stoch_signal(),
                         line=dict(color='blue', width=1),
                         name="stochastic_signal",
                        ), row=4, col=1)
fig.add_trace(go.Scatter(x=df.index,
                         y=stoch.stoch_signal(),
                         line=dict(color='blue', width=1),
                         name="stochastic_signal",
                        ), row=4, col=1)
# RSI
fig.add_trace(go.Scatter(x=df.index,
                         y=rsi,
                         line=dict(color='blue', width=2),
                         name="RSI",
                        ), row=5, col=1)



# update layout by changing the plot size, hiding legends & rangeslider, and removing gaps between dates
fig.update_layout(height=900, width=1200, 
                  showlegend=False, 
                  xaxis_rangeslider_visible=False,
                  xaxis_rangebreaks=[dict(values=dt_breaks)],
                  title="Playing with Fonts")

import plotly.express as px

fig = px.scatter(
        df, x="sepal_length", y="sepal_width", height=350,
        color="species", title="Playing with Fonts")

fig.update_layout(
    font_family="Courier New",
    font_color="blue",
    title_font_family="Times New Roman",
    title_font_color="red",
    legend_title_font_color="green"
)

# update y-axis label
fig.update_yaxes(title_text="Price", row=1, col=1)
fig.update_yaxes(title_text="Volume", row=2, col=1)
fig.update_yaxes(title_text="MACD", showgrid=True, row=3, col=1)
fig.update_yaxes(title_text="Stochastic", row=4, col=1)
fig.update_yaxes(title_text="RSI", row=5, col=1)
fig.update_yaxes(title_text="SMA200", row=6, col=1)

# fig.show()

# removing white spaces
fig.update_layout(margin=go.layout.Margin(
        l=20, #left margin
        r=20, #right margin
        b=20, #bottom margin
        t=20  #top margin
    ))



fig.show()

[HERE continue](https://stackoverflow.com/questions/71411995/pandas-plotly-secondary-graph-needs-to-be-to-rsi)
[second](https://www.marketcalls.in/python/using-plotly-and-pandas-ta-to-plot-technical-indicators-python-tutorial.html)

[third](https://github.com/topics/technical-indicators?o=desc&s=updated)

[PandasTA_Strategy_Examples.ipynb](https://github.com/twopirllc/pandas-ta/blob/main/examples/PandasTA_Strategy_Examples.ipynb)

In [ ]:
%pip install --user plotly.express==0.4.1
import plotly.express as px
df = px.data.iris()
fig = px.scatter(df, x="sepal_width", y="sepal_length", color="species")
fig.show()

# here continue 
# plotly.express px.data pandas

test